In [1]:
%cd ..

/volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark


In [2]:
# this just to make sure we are using only on CPU
import os
os.environ["CUDA_VISIBLE_DEVICES"]="-1"

In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib nbagg

import os.path as op
import time


from keras.callbacks import TensorBoard, ModelCheckpoint
from keras_tqdm import TQDMNotebookCallback
import numpy as np
import tensorflow as tf

from fastmri_recon.models.cascading import cascade_net
from fastmri_recon.data.oasis_sequences import Masked2DSequence

/volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/venv/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
# paths
train_path = '/media/Zaccharie/UHRes/OASIS_data/'

In [5]:
# generators
AF = 4
train_gen = Masked2DSequence(
    train_path, 
    af=AF, 
    inner_slices=32, 
    scale_factor=1e-2, 
    seed=0, 
    val_split=0.1,
)
val_gen = train_gen.val_sequence

train_gen.filenames = train_gen.filenames[:5]
val_gen.filenames = val_gen.filenames[:1]

In [6]:
run_params = {
    'n_cascade': 2, 
    'n_convs': 2, 
    'n_filters': 16,
    'noiseless': True,
}

n_epochs = 50
run_id = f'cascadenet_af{AF}_oasis_{int(time.time())}'
chkpt_path = f'checkpoints/{run_id}' + '-{epoch:02d}.hdf5'
print(run_id)

cascadenet_af4_oasis_1568894522


In [7]:
chkpt_cback = ModelCheckpoint(chkpt_path, period=500, save_weights_only=True)
log_dir = op.join('logs', run_id)
tboard_cback = TensorBoard(
    log_dir=log_dir, 
    histogram_freq=0, 
    write_graph=True, 
    write_images=False, 
)
tqdm_cb = TQDMNotebookCallback(metric_format="{name}: {value:e}")

In [8]:
model = cascade_net(input_size=(None, 256, 1), lr=1e-3, fastmri=False, **run_params)
print(model.summary(line_length=100))

W0919 14:02:03.115803 140371369891584 deprecation.py:323] From /volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/fastmri_recon/helpers/nn_mri.py:74: to_float (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.


Model: "model_1"
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
kspace_input (InputLayer)        (None, None, 256, 1)  0                                            
____________________________________________________________________________________________________
ifft_simple (Lambda)             (None, None, 256, 1)  0           kspace_input[0][0]               
____________________________________________________________________________________________________
lambda_1 (Lambda)                (None, None, 256, 1)  0           ifft_simple[0][0]                
____________________________________________________________________________________________________
lambda_2 (Lambda)                (None, None, 256, 1)  0           ifft_simple[0][0]                
__________________________________________________________________________

In [ ]:
%%time
model.fit_generator(
    train_gen, 
    steps_per_epoch=5, 
    epochs=n_epochs,
    validation_data=val_gen,
    validation_steps=1,
    verbose=0,
    callbacks=[tqdm_cb, tboard_cback, chkpt_cback, ],
#     max_queue_size=100,
    use_multiprocessing=True,
    workers=10,
    shuffle=False,
)

W0919 14:02:04.887423 140371369891584 deprecation_wrapper.py:119] From /volatile/home/Zaccharie/workspace/keras/keras/backend/tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.

W0919 14:02:04.902261 140371369891584 deprecation_wrapper.py:119] From /volatile/home/Zaccharie/workspace/keras/keras/callbacks/tensorboard_v1.py:200: The name tf.summary.merge_all is deprecated. Please use tf.compat.v1.summary.merge_all instead.

W0919 14:02:04.904093 140371369891584 deprecation_wrapper.py:119] From /volatile/home/Zaccharie/workspace/keras/keras/callbacks/tensorboard_v1.py:203: The name tf.summary.FileWriter is deprecated. Please use tf.compat.v1.summary.FileWriter instead.



/volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/fastmri_recon/data/oasis_sequences.py:103: ComplexWarning: Casting complex values to real discards the imaginary part
  images_scaled = images_scaled.astype(np.float32)
/volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/fastmri_recon/data/oasis_sequences.py:103: ComplexWarning: Casting complex values to real discards the imaginary part
  images_scaled = images_scaled.astype(np.float32)
/volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/fastmri_recon/data/oasis_sequences.py:103: ComplexWarning: Casting complex values to real discards the imaginary part
  images_scaled = images_scaled.astype(np.float32)
/volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/fastmri_recon/data/oasis_sequences.py:103: ComplexWarning: Casting complex values to real discards the imaginary part
  images_scaled = images_scaled.astype(np.float32)
/volatile/home/Zaccharie/workspace/fastmri-reproducible-benc

/volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/fastmri_recon/data/oasis_sequences.py:103: ComplexWarning: Casting complex values to real discards the imaginary part
  images_scaled = images_scaled.astype(np.float32)
/volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/fastmri_recon/data/oasis_sequences.py:103: ComplexWarning: Casting complex values to real discards the imaginary part
  images_scaled = images_scaled.astype(np.float32)
/volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/fastmri_recon/data/oasis_sequences.py:103: ComplexWarning: Casting complex values to real discards the imaginary part
  images_scaled = images_scaled.astype(np.float32)
/volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/fastmri_recon/data/oasis_sequences.py:103: ComplexWarning: Casting complex values to real discards the imaginary part
  images_scaled = images_scaled.astype(np.float32)
/volatile/home/Zaccharie/workspace/fastmri-reproducible-benc

/volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/fastmri_recon/data/oasis_sequences.py:103: ComplexWarning: Casting complex values to real discards the imaginary part
  images_scaled = images_scaled.astype(np.float32)
/volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/fastmri_recon/data/oasis_sequences.py:103: ComplexWarning: Casting complex values to real discards the imaginary part
  images_scaled = images_scaled.astype(np.float32)
/volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/fastmri_recon/data/oasis_sequences.py:103: ComplexWarning: Casting complex values to real discards the imaginary part
  images_scaled = images_scaled.astype(np.float32)
/volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/fastmri_recon/data/oasis_sequences.py:103: ComplexWarning: Casting complex values to real discards the imaginary part
  images_scaled = images_scaled.astype(np.float32)
/volatile/home/Zaccharie/workspace/fastmri-reproducible-benc

/volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/fastmri_recon/data/oasis_sequences.py:103: ComplexWarning: Casting complex values to real discards the imaginary part
  images_scaled = images_scaled.astype(np.float32)
/volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/fastmri_recon/data/oasis_sequences.py:103: ComplexWarning: Casting complex values to real discards the imaginary part
  images_scaled = images_scaled.astype(np.float32)
/volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/fastmri_recon/data/oasis_sequences.py:103: ComplexWarning: Casting complex values to real discards the imaginary part
  images_scaled = images_scaled.astype(np.float32)
/volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/fastmri_recon/data/oasis_sequences.py:103: ComplexWarning: Casting complex values to real discards the imaginary part
  images_scaled = images_scaled.astype(np.float32)
/volatile/home/Zaccharie/workspace/fastmri-reproducible-benc

...

/volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/fastmri_recon/data/oasis_sequences.py:103: ComplexWarning: Casting complex values to real discards the imaginary part
  images_scaled = images_scaled.astype(np.float32)
/volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/fastmri_recon/data/oasis_sequences.py:103: ComplexWarning: Casting complex values to real discards the imaginary part
  images_scaled = images_scaled.astype(np.float32)
/volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/fastmri_recon/data/oasis_sequences.py:103: ComplexWarning: Casting complex values to real discards the imaginary part
  images_scaled = images_scaled.astype(np.float32)
/volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/fastmri_recon/data/oasis_sequences.py:103: ComplexWarning: Casting complex values to real discards the imaginary part
  images_scaled = images_scaled.astype(np.float32)
/volatile/home/Zaccharie/workspace/fastmri-reproducible-benc

/volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/fastmri_recon/data/oasis_sequences.py:103: ComplexWarning: Casting complex values to real discards the imaginary part
  images_scaled = images_scaled.astype(np.float32)
/volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/fastmri_recon/data/oasis_sequences.py:103: ComplexWarning: Casting complex values to real discards the imaginary part
  images_scaled = images_scaled.astype(np.float32)
/volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/fastmri_recon/data/oasis_sequences.py:103: ComplexWarning: Casting complex values to real discards the imaginary part
  images_scaled = images_scaled.astype(np.float32)
/volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/fastmri_recon/data/oasis_sequences.py:103: ComplexWarning: Casting complex values to real discards the imaginary part
  images_scaled = images_scaled.astype(np.float32)
/volatile/home/Zaccharie/workspace/fastmri-reproducible-benc

/volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/fastmri_recon/data/oasis_sequences.py:103: ComplexWarning: Casting complex values to real discards the imaginary part
  images_scaled = images_scaled.astype(np.float32)
/volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/fastmri_recon/data/oasis_sequences.py:103: ComplexWarning: Casting complex values to real discards the imaginary part
  images_scaled = images_scaled.astype(np.float32)
/volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/fastmri_recon/data/oasis_sequences.py:103: ComplexWarning: Casting complex values to real discards the imaginary part
  images_scaled = images_scaled.astype(np.float32)
/volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/fastmri_recon/data/oasis_sequences.py:103: ComplexWarning: Casting complex values to real discards the imaginary part
  images_scaled = images_scaled.astype(np.float32)
/volatile/home/Zaccharie/workspace/fastmri-reproducible-benc

/volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/fastmri_recon/data/oasis_sequences.py:103: ComplexWarning: Casting complex values to real discards the imaginary part
  images_scaled = images_scaled.astype(np.float32)
/volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/fastmri_recon/data/oasis_sequences.py:103: ComplexWarning: Casting complex values to real discards the imaginary part
  images_scaled = images_scaled.astype(np.float32)
/volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/fastmri_recon/data/oasis_sequences.py:103: ComplexWarning: Casting complex values to real discards the imaginary part
  images_scaled = images_scaled.astype(np.float32)
/volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/fastmri_recon/data/oasis_sequences.py:103: ComplexWarning: Casting complex values to real discards the imaginary part
  images_scaled = images_scaled.astype(np.float32)
/volatile/home/Zaccharie/workspace/fastmri-reproducible-benc

/volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/fastmri_recon/data/oasis_sequences.py:103: ComplexWarning: Casting complex values to real discards the imaginary part
  """Get a training triplet from the file at `idx` in `self.filenames`.
/volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/fastmri_recon/data/oasis_sequences.py:103: ComplexWarning: Casting complex values to real discards the imaginary part
  """Get a training triplet from the file at `idx` in `self.filenames`.
/volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/fastmri_recon/data/oasis_sequences.py:103: ComplexWarning: Casting complex values to real discards the imaginary part
  """Get a training triplet from the file at `idx` in `self.filenames`.
/volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/fastmri_recon/data/oasis_sequences.py:103: ComplexWarning: Casting complex values to real discards the imaginary part
  """Get a training triplet from the file at `id

In [ ]:
# %%time
# # simple overfit trials
# data = train_gen[0]
# val_data = val_gen[1]
# model.fit(
#     x=data[0], 
#     y=data[1], 
# #     validation_data=val_data, 
#     batch_size=data[0][0].shape[0], 
# #     callbacks=[tqdm_cb, tboard_cback,],
#     epochs=500,
#     verbose=2, 
#     shuffle=False,
# )